# Phase II: Data Curation, Exploratory Analysis and Plotting (5\%)

### Team Members:
- Logan Lary
- Mark Tran
- Sabrina Valerjev

## Part 1: 
(1%) Expresses the central motivation of the project and explains the (at least) two key questions to be explored. Gives a summary of the data processing pipeline so a technical expert can easily follow along.

## Project Motivation

## Summary of the Data Processing Pipeline